# EfficientNet

## Installation and import of the libraries

In [10]:
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install tensorflow
!pip install efficientnet
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from tensorflow.keras import layers, models
import efficientnet.tfkeras as efn
from tensorflow.keras.preprocessing import image



## Preprocess

In [19]:
dtypes = {
    'cane_height_type': pd.Int64Dtype(),
    'closure_placement': pd.Int64Dtype(),
    'heel_shape_type': pd.Int64Dtype(),
    'knit_structure': pd.Int64Dtype(),
    'length_type': pd.Int64Dtype(),
    'neck_lapel_type': pd.Int64Dtype(),
    'silhouette_type': pd.Int64Dtype(),
    'sleeve_length_type': pd.Int64Dtype(),
    'toecap_type': pd.Int64Dtype(),
    'waist_type': pd.Int64Dtype(),
    'woven_structure': pd.Int64Dtype()
}
df = pd.read_csv("./archive/attribute_data_2.csv", dtype=dtypes)
product_data_df = pd.read_csv('./archive/product_data.csv')
df = df.merge(product_data_df[['cod_modelo_color', 'des_filename']], on='cod_modelo_color', how='left')
# img_dict = dict(zip(product_data_df['cod_modelo_color'], product_data_df['des_filename']))
# Assume the CSV has columns ['image_id', 'attribute', 'value']
# One-hot encode the 'value' column for each attribute
attribute_columns = ['silhouette_type', 'neck_lapel_type', 'woven_structure', 'knit_structure',
                     'heel_shape_type', 'length_type', 'sleeve_length_type', 'toecap_type',
                     'waist_type', 'closure_placement', 'cane_height_type']

base_path = "./archive/images/images/"
df = df[df['des_filename'].apply(lambda x: os.path.exists(os.path.join(base_path, x)))]
df.reset_index(drop=True, inplace=True)
df = df.head(16000)

In [8]:
start = 0
end = 1000
full_df = df

In [9]:
df = full_df[start:end]
start += 1000
end += 1000

In [20]:

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)



def load_and_preprocess_image(image_path, label):
    # Load and decode the image file
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Decode the JPEG image (if you have other formats, use decode_image)

    # Resize the image
    img = tf.image.resize(img, [224, 224])

    # Normalize the image
    img = img / 255.0  # Normalize to [0, 1]

    # Set the shape explicitly
    img.set_shape([224, 224, 3])

    return img, label

def create_dataset(df, image_folder, batch_size=16):
    # Convert filenames and labels into tensors
    image_paths = df['des_filename'].apply(lambda x: os.path.join(image_folder, x)).values
    labels = df[[
        'silhouette_type',
        'waist_type',
        'neck_lapel_type',
        'sleeve_length_type',
        'toecap_type',
        'closure_placement',
        'cane_height_type',
        'heel_shape_type',
        'knit_structure',
        'length_type',
        'woven_structure'
    ]].fillna(-1).values  # Replace NaNs with -1 if necessary

    # Convert to TensorFlow tensors
    image_paths = tf.convert_to_tensor(image_paths, dtype=tf.string)
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)

    # Create a tf.data.Dataset from image paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    # Apply the preprocessing function to load and preprocess images
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

    # Batch the data and prefetch for performance
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset

# Create the train and test datasets
train_dataset = create_dataset(train_df, './archive/images/images/')
test_dataset = create_dataset(test_df, './archive/images/images/')

## Build

In [21]:
# Build the EfficientNet model
base_model = efn.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Create the model architecture
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(len(attribute_columns), activation='sigmoid')(x)  # One output per attribute

# Build and compile the model
model = models.Model(inputs, outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


## Train

In [18]:
train_dataset

<_ParallelMapDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>

In [22]:
history = model.fit(
    train_dataset,
    epochs=5,
    validation_data=test_dataset
)

Epoch 1/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 242s 297ms/step - accuracy: 0.6858 - loss: -94.5623 - val_accuracy: 0.7131 - val_loss: -382.9436
Epoch 2/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 232s 291ms/step - accuracy: 0.7226 - loss: -470.7144 - val_accuracy: 0.7131 - val_loss: -767.8943
Epoch 3/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 231s 289ms/step - accuracy: 0.7226 - loss: -847.9926 - val_accuracy: 0.7134 - val_loss: -1153.4126
Epoch 4/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 233s 291ms/step - accuracy: 0.7231 - loss: -1228.0515 - val_accuracy: 0.7128 - val_loss: -1539.2028
Epoch 5/5
800/800 ━━━━━━━━━━━━━━━━━━━━ 233s 291ms/step - accuracy: 0.7225 - loss: -1606.2124 - val_accuracy: 0.7131 - val_loss: -1924.7952


## Test

In [35]:
test_df = pd.read_csv("./archive/test_data.csv", dtype=dtypes)[['cod_modelo_color', 'des_filename']]
test_df = test_df.head(100)

In [57]:
# Function to make predictions in batches

# Function to preprocess images, similar to training pipeline
def load_and_preprocess_image(image_path):
    # Read and decode the image
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Decode image to RGB
    img = tf.image.resize(img, [224, 224])  # Resize image to 224x224
    img = img / 255.0  # Normalize the image to [0, 1]
    
    img.set_shape([224, 224, 3])  # Explicitly set shape to avoid shape issues
    return img

def predict_in_batches(df, image_folder, model, batch_size=50):
    # Extract image paths from the dataframe
    image_paths = df['des_filename'].apply(lambda x: os.path.join(image_folder, x)).values

    # Create a TensorFlow Dataset from image paths
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    
    # Map the image paths to their respective preprocessed images
    dataset = dataset.map(lambda path: load_and_preprocess_image(path), num_parallel_calls=tf.data.AUTOTUNE)
    
    # Batch the dataset and prefetch for performance
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    # Initialize an empty list to store predictions
    predictions = []

    # Loop through the dataset and make predictions
    for batch_images in dataset:
        batch_predictions = model.predict(batch_images)
        predictions.extend(batch_predictions)  # Collect all predictions

    # Convert predictions to a DataFrame for easier interpretation
    return predictions

# Predict on the batch of images
predictions = predict_in_batches(test_df, './archive/images/images/', model)



2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 289ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 281ms/step


In [58]:
predictions

[array([1.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 5.6670587e-34, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 1.0000000e+00, 0.0000000e+00], dtype=float32),
 array([1.0000000e+00, 0.0000000e+00, 1.0000000e+00, 7.5365602e-26,
        0.0000000e+00, 1.6323925e-31, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 1.0000000e+00, 0.0000000e+00], dtype=float32),
 array([1.000000e+00, 0.000000e+00, 1.000000e+00, 1.000000e+00,
        0.000000e+00, 5.836386e-20, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 1.000000e+00, 0.000000e+00], dtype=float32),
 array([1.000000e+00, 0.000000e+00, 1.000000e+00, 1.000000e+00,
        0.000000e+00, 4.138966e-08, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 1.000000e+00, 0.000000e+00], dtype=float32),
 array([1.00000e+00, 0.00000e+00, 1.00000e+00, 1.00000e+00, 0.00000e+00,
        3.96242e-20, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
        0.00000e+00], dtype=float32),
 array([1.0000